In [1]:
import duckdb
import pandas as pd
import os
from lotus.dtype_extensions import ImageArray
from lotus.types import CascadeArgs, ProxyModel
import pandas as pd
from join_optimizer.join_optimizer.productDisplayName_index.evaluate import evaluate_filter


parquet_path_sampeled = 'fashion_product_images/styles.parquet'
parquet_path = 'fashion_product_images/styles.parquet'
details_path = 'fashion_product_images/styles_details.parquet'
sample_size = 0.02
df = duckdb.query(f"""
with images as (
    SELECT *
    FROM parquet_scan('{parquet_path}')
    USING SAMPLE {sample_size * 100} PERCENT (reservoir, 80)
    )
    select
     images.id ,images.subcategory, images.articletype, images.basecolour, details.price, images.productDisplayName,
     -- styleimages.default.resolutions."360X480"  as imageURL
     styleimages.default.imageURL  as imageURL
    -- *
    from images, parquet_scan('{details_path}') details
    where images.id = details.id
    -- and details.price <1000
    order by images.id
""").to_df()



/u/halle/bouassid/home_at/miniconda3/envs/lotus/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
image_file_names = df["id"]
image_URLs = df["imageURL"]
image_paths = [os.path.join("fashion_product_images/images_resized", str(image) + ".jpg") for image in image_file_names]
df2 = pd.DataFrame({"image": ImageArray(image_URLs), "label": image_file_names, "image_path": image_paths, "image_URLs": image_URLs , "articleType": df["articleType"], "baseColour": df["baseColour"]})
merged_df = pd.merge(df, df2,  left_on='id', right_on='label')
merged_df.columns = ['_' + col for col in merged_df.columns]


In [ ]:
x.p


In [ ]:

import lotus
from lotus.models import LM, SentenceTransformersRM
from lotus.types import CascadeArgs
from lotus.vector_store import FaissVS
lm = LM(model="gemini/gemini-2.0-flash-lite")
rm = SentenceTransformersRM(model="intfloat/e5-base-v2")
vs = FaissVS()
lotus.settings.configure(lm=lm, rm=rm, vs=vs)

lm.print_total_usage()

filtered_df2 = (df2
                .sem_filter("the content  of {image} shows a white t-shirt")
                )
lm.print_total_usage()


In [ ]:
image_paths_big = [os.path.join("fashion_product_images/images", str(image) + ".jpg") for image in image_file_names if str(image)[10] == 1]
df2_big = pd.DataFrame({"image": ImageArray(image_paths_big), "label": image_file_names, "image_path": image_paths})

lm_2 = LM(model="gemini/gemini-2.0-flash-lite")
rm = SentenceTransformersRM(model="intfloat/e5-base-v2")
vs = FaissVS()
lotus.settings.configure(lm=lm_2, rm=rm, vs=vs)

lm_2.print_total_usage()

filtered_df2_big = (df2_big
                .sem_filter("the content  of {image} shows a white t-shirt")
                )
lm_2.print_total_usage()


In [ ]:
filtered_df2


In [ ]:
filtered_df = (df
                .sem_filter("The {articleType}, {baseColour} and {productDisplayName} might show a t-shirt that might be white")
                )
lm.print_total_usage()



In [ ]:
filtered_df


In [ ]:
expr = "Does {productDisplayName} exactly and precisely match the {image}?"


cascade_args = CascadeArgs(recall_target=0.8, precision_target=0.8)
res = filtered_df.sem_join(filtered_df2, expr, return_stats=True, strategy="zs-cot")


print(f"Joined {df.shape[0]} rows from df1 with {filtered_df2.shape[0]} rows from df2")
# print(f"    Join cascade took {stats['join_resolved_by_large_model']} LM calls")
# print(f"    Helper resolved {stats['join_resolved_by_helper_model']} LM calls")
# print(f"Join cascade used {stats['total_LM_calls']} LM calls in total")
# print(f"Naive join would require {df.shape[0]*df2.shape[0]} LM calls")
res

In [ ]:
lm.print_total_usage()


In [ ]:
expr = "given {productDisplayName}, which {image} does more exactly and precisely match?"


ranked, stats = res.sem_topk(
    expr,
    K = 1,
    group_by=["productDisplayName"],
    method="quick",
    return_stats=True
)
ranked

In [ ]:
lm.print_total_usage()


# SEM INDEX


#### Creating the index

In [3]:
from lotus.vector_store import FaissVS
import lotus
from lotus.models import LM, SentenceTransformersRM

gpt_4o_mini = LM("gpt-4o-mini")
gpt_4o = LM("gpt-4o")

# CLIP embedding model – works for both text & image
# rm  = SentenceTransformersRM(model="clip-ViT-B-32")
rm  = SentenceTransformersRM(model="clip-ViT-L-14", max_batch_size=32)

lotus.settings.configure(lm=gpt_4o, helper_lm=gpt_4o_mini, rm=rm, vs=FaissVS())

2025-08-15 18:11:50,967 - INFO - Use pytorch device_name: cpu
2025-08-15 18:11:50,968 - INFO - Load pretrained SentenceTransformer: clip-ViT-L-14
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
merged_df = merged_df.sem_index("_image", index_dir=f"image_{sample_size}_index")
df = df.sem_index("productDisplayName", index_dir=f"productDisplayName_{sample_size}_index")



In [4]:
merged_df = merged_df.load_sem_index("_image", index_dir=f"image_{sample_size}_index")
merged_df = merged_df.load_sem_index("_imageURL", index_dir=f"image_{sample_size}_index")
df = df.load_sem_index("productDisplayName", index_dir=f"productDisplayName_{sample_size}_index")
# merged_df = merged_df.sem_map("extract the url from {_default} of an image resolution that is not sooo big but still okay for an llm to recognize the details. 360X480 or the one just bigger.", suffix = "_imageURL")


#### Full LLM calls

In [5]:
# merged_df_filtered_white_tshirts = merged_df.sem_filter("{_image} is a product of white T-shirt", return_stats=False)
# merged_df_filtered_socks = merged_df.sem_filter("{_image} is a product of a sock", return_stats=False)
# merged_df_filtered_wallet = merged_df.sem_filter("{_image} shows a wallet", return_stats=False)
merged_df_filtered_black_footwear = merged_df.sem_filter("You will receive an {_productDisplayName} of a product. Determine whether the product can be worn on the feet, like shoes, sandals, flip-flops, ... The predominant color of the depicted product should be black. If there are multiple products in the picture, always refer to the most prominent one.", return_stats=False)


Filtering:   0%|           0/888 LM calls [00:00<?, ?it/s]2025-08-15 18:12:12,066 - INFO - Retrying request to /chat/completions in 0.485447 seconds
Filtering: 100%|██████████ 888/888 LM calls [00:10<00:00, 83.04it/s]


##### Binary search filter

In [ ]:

cascade_args = CascadeArgs(
    recall_target=0.9,
    precision_target=0.9,
    sampling_percentage=0.1,
    proxy_model=ProxyModel.EMBEDDING_MODEL,
)

merged_df_filtered =  merged_df.sem_filter("the main product in the {_image} is a watch.",cascade_args=cascade_args, return_stats=True, find_top_k=True)


In [ ]:
from join_optimizer.join_optimizer.productDisplayName_index.evaluate import *

metrics, FP, FN = evaluate_filter(
    dataset_df=merged_df_filtered_black_footwear,
    filtered_df=merged_df_filtered,
    article_type=None,
    base_colour=None
)
print(metrics)

#### Sampling

In [ ]:
import sys
import importlib
%load_ext autoreload
%autoreload 2

importlib.reload(sys.modules['lotus.sem_ops.cascade_utils'])
importlib.reload(sys.modules['lotus.sem_ops.sem_filter'])
importlib.reload(sys.modules['lotus'])
importlib.reload(sys.modules['join_optimizer.join_optimizer.productDisplayName_index.evaluate'])
importlib.reload(sys.modules['pandas'])
import lotus.sem_ops.sem_filter
import lotus

# Ensure you import the module (not just the function) so autoreload can update it:
import lotus.sem_ops.sem_filter as sem_filter_mod


In [ ]:
cascade_args = CascadeArgs(
    recall_target=0.95,
    precision_target=0.9,
    sampling_percentage=0.1,
    proxy_model=ProxyModel.EMBEDDING_MODEL,
    cascade_IS_weight=1,
    cascade_num_calibration_quantiles = 100,
    failure_probability=0.1,
    cascade_IS_random_seed=114,
    cascade_IS_max_sample_range=444

)

merged_df_filtered_big =  merged_df.sem_filter("{_image} shows a wallet", cascade_args=cascade_args ,return_stats=False, find_top_k=False)


In [ ]:
from join_optimizer.join_optimizer.productDisplayName_index.evaluate import *

metrics, FP, FN = evaluate_filter(
    dataset_df=merged_df_filtered_wallet,
    filtered_df=merged_df_filtered_big,
    article_type=None,
    base_colour=None
)
print(metrics)

#### Just sim_search

In [ ]:
# sem_search with similarity scores returned
sim_df_with_scores = merged_df.sem_search(
    "_imageURL",
    # "You will receive an image of a product. Determine whether the product can be worn on the feet, like shoes, sandals, flip-flops, ... The predominant color of the depicted product should be black. If there are multiple products in the picture, always refer to the most promiment one.",
    "_imageURL the main product in the image is a watch.",
    K=60,
    return_scores=True,
    suffix="_similarity_score"
)


#### Joins

In [ ]:
res_sim_join = merged_df_filtered.sem_sim_join(df, left_on='_image', right_on='productDisplayName', K=1, keep_index=True)

In [ ]:
expr = "Does {productDisplayName} exactly and precisely match the {_image}?"
from lotus.types import CascadeArgs, ProxyModel

cascade_args = CascadeArgs(
    recall_target=0.9,
    precision_target=0.75,
    sampling_percentage=0.04,
    proxy_model=ProxyModel.EMBEDDING_MODEL,
)

res = merged_df_filtered.sem_join(df, expr,cascade_args=cascade_args, return_stats=True)

# print(f"Joined {df.shape[0]} rows from df1 with {filtered_df2.shape[0]} rows from df2")
# print(f"    Join cascade took {stats['join_resolved_by_large_model']} LM calls")
# print(f"    Helper resolved {stats['join_resolved_by_helper_model']} LM calls")
# print(f"Join cascade used {stats['total_LM_calls']} LM calls in total")
# print(f"Naive join would require {df.shape[0]*df2.shape[0]} LM calls")


In [ ]:
res = 0

In [ ]:

expr = "given {productDisplayName}, which {_image} does more exactly and precisely match?"

ranked, stats = res[0].sem_topk(
    expr,
    K=1,
    group_by=["productDisplayName"],
    method="quick",
    return_stats=True
)
ranked


In [ ]:
join_topk = ranked.query('_id == id')
sim_1 = res_sim_join.query('_id == id')